# Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

* Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
* Crear un modelo para predecir el volumen de reservas en pozos nuevos.
* Elegir los pozos petrolíferos que tienen los valores estimados más altos.
* Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

---

# Condiciones:
* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

---

Descripción de datos
Los datos de exploración geológica de las tres regiones se almacenan en archivos:

* /datasets/geo_data_0.csv. Descarga el conjunto de datos
* /datasets/geo_data_1.csv. Descarga el conjunto de datos
* /datasets/geo_data_2.csv. Descarga el conjunto de datos
* id — identificador único de pozo de petróleo
* f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
* product — volumen de reservas en el pozo de petróleo (miles de barriles).

---

# Instrucciones del proyecto
1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:
    1. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
    2. Entrena el modelo y haz predicciones para el conjunto de validación.
    3. Guarda las predicciones y las respuestas correctas para el conjunto de validación.
    4. Muestra el volumen medio de reservas predicho y RMSE del modelo.
    5. Analiza los resultados.
    6. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.
3. Prepárate para el cálculo de ganancias:
    1. Almacena todos los valores necesarios para los cálculos en variables separadas.
    2. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
    3. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.
4. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:
    1. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').
    2. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.
    3. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.
5. Calcula riesgos y ganancias para cada región:
    1. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.
    2. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.
    3. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Preparación de los datos

In [2]:
url_0 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_0.csv'
url_1 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_1.csv'
url_2 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_2.csv'

In [3]:
df_0 = pd.read_csv(url_0)
df_1 = pd.read_csv(url_1)
df_2 = pd.read_csv(url_2)

In [4]:
def explorar_dataset(df):
    print('---' * 10, '\n', f'Información del dataframe')
    print(df.info())
    print('---' * 10, '\n', f'Dimensiones del dataframe')
    print(df.shape)
    print('---' * 10, '\n', f'Primeras filas del dataframe')
    print(df.head(15))
    print('---' * 10, '\n', f'Descripción del dataframe')
    print(df.describe())
    print('---' * 10, '\n', f'Valores nulos del dataframe')
    print(df.isnull().sum())
    print('---' * 10, '\n', f'Valores duplicados del dataframe')
    print(df.duplicated().sum())
    print('---' * 10)

## Dataframe 0

In [5]:
explorar_dataset(df_0)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   txEyH  0.705745 -0.497823  1.221170  105.280062
1   2acmU  1.334711 -0.340164  4.365080   73.037750
2   409Wp  1.022732  0.151990  1.419926   85.265647
3   iJLyR -0.032172  0.139033  2.978566  168.620776
4   Xdl7t  1.988431  0.155413  4.751769  154.036647
5   wX4Hy  0.969570  0.489775 -0.735383   64.741541
6   t

## Dataframe 1

In [6]:
explorar_dataset(df_1)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id         f0         f1        f2     product
0   kBEdx -15.001348  -8.276000 -0.005876    3.179103
1   62mP7  14.272088  -3.475083  0.999183   26.953261
2   vyE1P   6.263187  -5.948386  5.001160  134.766305
3   KcrkZ -13.081196 -11.506057  4.999415  137.945408
4   AHL4O  12.702195  -8.147433  5.004363  134.766305
5   HHckp  -3.327590  -2.205276  3.003647   8

## Dataframe 2

In [7]:
explorar_dataset(df_2)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   fwXo0 -1.146987  0.963328 -0.828965   27.758673
1   WJtFt  0.262778  0.269839 -2.530187   56.069697
2   ovLUW  0.194587  0.289035 -5.586433   62.871910
3   q6cA6  2.236060 -0.553760  0.930038  114.572842
4   WPMUX -0.515993  1.716266  5.899011  149.600746
5   LzZXx -0.758092  0.710691  2.585887   90.222465
6   W

# Entrenar el modelo para cada Región

In [14]:
def entrenar_modelo(df):
    """
    Función que divide los datos en un conjunto de entrenamiento y un conjunto
    de validación en una proporción de 75:25. Entrena el modelo y hace predicciones
    para el conjunto de validación. Guarda las predicciones y las respuestas correctas
    para el conjunto de validación. Muestra el volumen medio de reservas predicho
    y RMSE del modelo.

    Parámetros
        df: DataFrame con los datos de entrenamiento

    Retorna
        predicted_valid: predicciones del modelo
        target_valid: respuestas correctas del modelo
        mean_reserves: volumen medio de reservas predicho
        rmse: RMSE del modelo
    """

    features_train, features_valid, target_train, target_valid = train_test_split(df, df['product'], test_size=0.25)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)

    mean_reserves = round(predicted_valid.mean(), 3)
    rmse = root_mean_squared_error(target_valid, predicted_valid)

    print(f'Volumen medio de reservas: {mean_reserves}')
    print(f'RMSE: {round(rmse, 3)}')

    return predicted_valid, target_valid, mean_reserves, rmse

In [15]:
df_0_final = df_0.drop(['id'], axis=1)
df_1_final = df_1.drop(['id'], axis=1)
df_2_final = df_2.drop(['id'], axis=1)

## Entrenamiento del modelo para la Región 0

In [16]:
region_0_predicted, region_0_target, region_0_mean, region_0_rmse = entrenar_modelo(df_0_final)

Volumen medio de reservas: 92.98
RMSE: 0.0


## Entrenamiento del modelo para la Región 1

In [11]:
region_1_predicted, region_1_target, region_1_mean, region_1_rmse = entrenar_modelo(df_1_final, 12345)

Volumen medio de reservas: 68.723
RMSE: 0.0


## Entrenamiento del modelo para la Región 2

In [12]:
region_2_predicted, region_2_target, region_2_mean, region_2_rmse = entrenar_modelo(df_2_final, 12345)

Volumen medio de reservas: 94.884
RMSE: 0.0


# Preparación para cálculo de ganancias